In [1]:
from flask import Flask, redirect, url_for, session, flash, render_template, request
from datetime import timedelta
from flask_sqlalchemy import SQLAlchemy

In [ ]:
app = Flask(__name__)
app.secret_key = 'hello'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
app.permanent_session_lifetime = timedelta(minutes=5)

db = SQLAlchemy(app)

class Users(db.Model):
    _id = db.Column("id", db.Integer, primary_key=True)
    name = db.Column(db.String(100))
    email = db.Column(db.String(100))

    def __init__(self, name, email):
        self.name = name
        self.email = email

@app.route("/")
def home():
    return "Hello"

@app.route("/view", methods=['POST', 'GET'])
def view():   
    if request.method == 'POST':
        user_to_delete = request.form['del_querry']
        Users.query.filter_by(name=user_to_delete).delete()
        db.session.commit()
        flash("User has been deleted")
        return redirect(url_for('view'))
    return render_template('view.html', values=Users.query.all())

@app.route("/login", methods=["POST","GET"])
def login():
    if request.method =="POST":
        user = request.form['nm']
        session.permanent = True
        session['user'] = user
        
        found_user = Users.query.filter_by(name=user).first()
        if found_user:
            session['email'] = found_user.email
        else:
            usr = Users(user, "")
            db.session.add(usr)
            db.session.commit()

        flash("login successful")
        return redirect(url_for("user"))
    else:
        if 'user' in session:
            flash("Already Logged in!!")
            return redirect(url_for('user'))

        return render_template('login1.html')

@app.route("/user", methods=["POST", "GET"])
def user():
    email = None
    if "user" in session:
        user =  session['user']

        if request.method == 'POST':
            email = request.form["email"]
            session['email'] = email
            found_user = Users.query.filter_by(name=user).first()
            found_user.email = email
            db.session.commit()
            flash("Email was saved")
        else:
            if 'email' in session:
                email = session['email']

        return render_template("user1.html", email = email)
    else:
        flash("you are not logged in!")
        return redirect(url_for('login'))

@app.route("/logout")
def logout():
    if 'user' in session:
        flash("you have been logged out!!", "info")
    session.pop("user", None)
    session.pop("email", None)
    return redirect(url_for("login"))

if __name__ == "__main__":
    with app.app_context():
        db.create_all()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Nov/2025 12:58:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 12:58:50] "GET /view HTTP/1.1" 200 -


In [16]:
# Here's how I made the deletion part:
# In view.html make a new form
#     <form action="#" method="POST">
#         <input type="text" name="del_querry" required="True"/>
#         <input type="submit" value="Delete"/>
#     </form>

# and added this to the view app route:
#     if request.method == 'POST':
#         user_to_delete = request.form.get('del_querry')
#         users.query.filter_by(name=user_to_delete).delete()
#         db.session.commit()